In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AI TOOLS/housing.csv")
# checking for null values
data.isnull().mean() * 100

longitude             0.000000
latitude              0.000000
housing_median_age    0.000000
total_rooms           0.000000
total_bedrooms        1.002907
population            0.000000
households            0.000000
median_income         0.000000
median_house_value    0.000000
ocean_proximity       0.000000
dtype: float64

In [ ]:
# handling null values in total_bedrooms with the most frequent value in respective column
data["total_bedrooms"].fillna(data['total_bedrooms'].mode()[0],inplace=True)
#checking the null values handled or not
data["total_bedrooms"].isnull().mean() * 100

0.0

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [3]:
print(data['ocean_proximity'].unique())

['NEAR BAY' '<1H OCEAN' 'INLAND' 'NEAR OCEAN' 'ISLAND']


In [4]:
print(data['ocean_proximity'].value_counts())

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64


In [5]:
#we need to convert categorical values by label encoding
# there are more than two categories, we have to use onehot encoding
ocean_prox_df = pd.get_dummies(data['ocean_proximity'],drop_first=True,dtype=int)
ocean_prox_df.head()

,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0


In [6]:
old_data = data.copy()
data.drop(['ocean_proximity','longitude','latitude'],axis=1,inplace=True)
data.head()

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [7]:
data = pd.concat([data,ocean_prox_df],axis=1)
data.head()

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0,0,1,0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0,0,1,0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0,0,1,0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0,0,1,0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0,0,1,0


In [18]:
#Split the data
from sklearn.model_selection import train_test_split
# split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(data.drop('median_house_value',axis=1),
data['median_house_value'], test_size=0.30,
random_state=101)

a. Linear Regression with Polynomial Features

In [24]:
#Impute missing values using SimpleImputer
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean') # You can choose a different strategy if needed

# Fit and transform the imputer on your training data
X_train_imputed = imputer.fit_transform(X_train)

# Transform the test data using the fitted imputer
X_test_imputed = imputer.transform(X_test) # Apply the same imputation to test data

# Now use the imputed data for PolynomialFeatures
X_ = poly.fit_transform(X_train_imputed)
Y_ = poly.fit_transform(y_train.values.reshape(-1,1))

# Continue with the rest of your code...
model.fit(X_,Y_)

#preparing test data for predictions
testX = poly.fit_transform(X_test_imputed) # Use imputed test data
# predicting the output for test data
predicted = model.predict(testX)
# expected output for test data
expected = poly.fit_transform(y_test.values.reshape(-1,1))
from sklearn.metrics import r2_score
r2 = r2_score(expected, predicted)
print('r2 score is', r2)

r2 score is -1.164625743332833


In [25]:
# example of calculate the root mean squared error
from sklearn.metrics import mean_squared_error
# calculate errors
errors = mean_squared_error(expected, predicted, squared=False)
# report error
print("root mean square error is :",errors)

root mean square error is : 1.4953564526118596e+16


b. Random Forest Regressor

In [27]:
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
# create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 101)
# fit the regressor with x and y data
regressor.fit(X_train_imputed, y_train) # Use the imputed training data

RandomForestRegressor(random_state=101)

In [ ]:
# test the output by changing values
predicted = regressor.predict(X_test)
expected = y_test
from sklearn.metrics import r2_score
r2 = r2_score(expected, predicted)
print('r2 score is', r2)

r2 score is 0.7055448208376878


In [ ]:
# example of calculate the root mean squared error
from sklearn.metrics import mean_squared_error
# calculate errors
errors =mean_squared_error(expected, predicted,squared=False)
# report error
print("root mean square error is :",errors)

root mean square error is : 63124.26671801734
